In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, StratifiedKFold, KFold, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
import category_encoders as ce
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler

In [38]:
df_train = pd.read_csv('Data_train.csv')
train = df_train.copy(deep=True)
df_test = pd.read_csv('Data_test.csv')
test = df_test.copy(deep=True)

Task1

In [39]:
print(df_train.isnull().sum())
print()
print(df_test.isnull().sum())

type                   0
group                 20
education              0
meal                   0
preparation course     0
score-1                0
score-2                0
score-3                0
dtype: int64

type                   0
group                 12
education              0
meal                   0
preparation course     0
dtype: int64


In [40]:
df_train['group'].fillna('unique', inplace=True)
df_test['group'].fillna('unique', inplace=True)

/var/folders/vv/nt_93ffn49329l9mqpwy9z0h0000gn/T/ipykernel_46335/3866996808.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['group'].fillna('unique', inplace=True)
/var/folders/vv/nt_93ffn49329l9mqpwy9z0h0000gn/T/ipykernel_46335/3866996808.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

Task 2

In [41]:
df_train['Pass'] = ((df_train['score-1'] > 50) &(df_train['score-2'] > 50) &
        (df_train['score-3'] > 50)).astype(int)
(df_train['Pass'] == 0).sum()

145

Task 3

№1)

In [42]:
wild = (df_train[df_train['type'] == 'wild']['Pass'] == 1).sum() / len(df_train[df_train['type'] == 'wild'])
dom = (df_train[df_train['type'] == 'domestic']['Pass'] == 1).sum() / len(df_train[df_train['type'] == 'domestic'])

In [43]:
print(round(abs(wild - dom), 2))
print(abs(wild - dom))

0.02
0.022580460755730947


№2

In [44]:
df_train[(df_train['Pass'] == 0) & (df_train['education'] == 'high school')].shape[0]

35

№3

In [45]:
df_train[(df_train['Pass'] == 1) & (df_train['preparation course'] == 'none') & (df_train['type'] == 'wild')].shape[0]

152

№4

In [46]:
df_train['score-1'].median()

66.0

№5

In [47]:
df_train[(df_train['type'] == 'domestic') & (df_train['preparation course'] == 'none')].describe()

,score-1,score-2,score-3,Pass
count,230.000000,230.000000,230.000000,230.000000
mean,61.373913,69.708696,68.465217,0.756522
std,16.085992,15.172585,15.477159,0.430117
min,0.000000,17.000000,10.000000,0.000000
25%,51.000000,60.250000,59.000000,1.000000
50%,62.000000,71.000000,70.000000,1.000000
75%,73.000000,80.000000,79.000000,1.000000
max,100.000000,100.000000,100.000000,1.000000


# Task 4


a)

In [48]:
cat_cols = ['type', 'group', 'education', 'meal', 'preparation course']
encoder = ce.OneHotEncoder(cols=cat_cols, use_cat_names=True)
df_train = encoder.fit_transform(df_train)
df_test = encoder.fit_transform(df_test)

тупо можно посмотерть на кол-во колонок в изначально датасете
и потом отнять кол-во колонок в датасете после аун хот енкоудинга

In [49]:
X = df_train.drop(columns=['Pass', 'score-1', 'score-2', 'score-3'])
y = df_train['Pass']

b)

In [50]:
model = DecisionTreeClassifier(max_depth=5, criterion='entropy', splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)
score = cross_val_score(model, X, y, cv=3, scoring='roc_auc')

In [51]:
round(score.mean(), 1)

0.7

# Task 5

a)

In [52]:
modelcv = DecisionTreeClassifier()
params = {'max_depth': np.arange(2, 20, 1)}
best = GridSearchCV(estimator=modelcv, cv = 3, param_grid=params, scoring='roc_auc')
best.fit(X, y)
print(best.best_params_)

{'max_depth': 2}


b)
тут у меня train/test это все теже train_df/test_df

train = df_train.copy(deep=True) - т.е по сути это train_df скопированный в другую фичу

я это сделал чисто что бы была чистая версия датасета(до всех изменений по заданию)



In [53]:
train['group'].fillna('unique', inplace=True)
test['group'].fillna('unique', inplace=True)
train['Pass'] = ((train['score-1'] > 50) &(train['score-2'] > 50) &
                    (train['score-3'] > 50)).astype(int)

train['cat_bio'] = train['type'] + ',' + train['group']


/var/folders/vv/nt_93ffn49329l9mqpwy9z0h0000gn/T/ipykernel_46335/3525902584.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['group'].fillna('unique', inplace=True)
/var/folders/vv/nt_93ffn49329l9mqpwy9z0h0000gn/T/ipykernel_46335/3525902584.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [54]:
cat_cols = ['education', 'meal', 'preparation course', 'cat_bio']
encoder = ce.OneHotEncoder(cols=cat_cols, use_cat_names=True)
train = encoder.fit_transform(train)


In [55]:
X = train.drop(columns=['Pass', 'score-1', 'score-2', 'score-3', 'type', 'group'])
y = train['Pass']

In [56]:
model = DecisionTreeClassifier(max_depth=5, criterion='entropy', splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

score = cross_val_score(model, X, y, cv=3, scoring='roc_auc')
round(score.mean(), 2)

0.68

Task 6

In [57]:
train = pd.read_csv('Data_train.csv')
test = pd.read_csv('Data_test.csv')
train['group'].fillna('unique', inplace=True)
train['group'].fillna('unique', inplace=True)

/var/folders/vv/nt_93ffn49329l9mqpwy9z0h0000gn/T/ipykernel_46335/3096881653.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['group'].fillna('unique', inplace=True)


In [58]:
cat_cols = train.select_dtypes(include=['object', 'category']).columns
for cols in cat_cols:
    print(f'Column {cols} unique values = {train[cols].unique()}')

Column type unique values = ['wild' 'domestic']
Column group unique values = ['group B' 'group C' 'group E' 'group D' 'group A' 'unique']
Column education unique values = ['some high school' "bachelor's degree" 'some college'
 "associate's degree" 'high school' "master's degree"]
Column meal unique values = ['standard' 'reduced']
Column preparation course unique values = ['completed' 'none']


Заенкодил категориальные данные маппингом: для фичей где всего присутствует всего 2 уникальных значения-бинарная мапа = [0.1]

для education установил иерархическую мапу от "низшего" к "высшему образованию"

а фичу group решил скормить XGboost так как он может рабоать с категориальными данным(там под капотом вроде TargetEncoder)

In [59]:
train['type'] = train['type'].map({'wild':0, 'domestic':1})
test['type'] = test['type'].map({'wild':0, 'domestic':1})

train['education'] = train['education'].map({'some high school':0, 'some college':0, 'high school':1,
                                             "associate's degree": 2, "bachelor's degree": 3, "master's degree": 4})
test['education'] = test['education'].map({'some high school':0, 'some college':0, 'high school':1,
                                             "associate's degree": 2, "bachelor's degree": 3, "master's degree": 4})

train['meal'] = train['meal'].map({'reduced': 0, 'standard': 1})
test['meal'] = test['meal'].map({'reduced': 0, 'standard': 1})

train['preparation course'] = train['preparation course'].map({'none': 0, 'completed': 1})
test['preparation course'] = test['preparation course'].map({'none': 0, 'completed': 1})

In [60]:
train['Pass'] = ((train['score-1'] > 50) &(train['score-2'] > 50) &
                    (train['score-3'] > 50)).astype(int)

In [61]:
for col in train.columns:
    if train[col].dtype == 'object':
        train[col] = train[col].astype('category')
print(train.dtypes)
for col in test.columns:
    if test[col].dtype == 'object':
        test[col] = test[col].astype('category')
print(test.dtypes)

type                     int64
group                 category
education                int64
meal                     int64
preparation course       int64
score-1                  int64
score-2                  int64
score-3                  int64
Pass                     int64
dtype: object
type                     int64
group                 category
education                int64
meal                     int64
preparation course       int64
dtype: object


In [62]:
X = train.drop(columns=['Pass', 'score-1', 'score-2', 'score-3'])
y = train['Pass']

У нас вроде бы есть дисбаланс классов для таргет фичи Pass

поэтому библа imblearn для оверсемплинга

In [63]:
sampler = RandomOverSampler()
X, y = sampler.fit_resample(X, y)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

для метрики ROC-AUC надо предсказывать вероятности принадлежности к классам, а не четкие предсказанные метки классов

predict_proba - для вероятностей, predict - классы в задачах классификации

In [64]:
model = XGBClassifier(enable_categorical=True)
model.fit(x_train, y_train)
proba = model.predict_proba(x_test)
roc_auc = roc_auc_score(y_test, proba[:, 1])
print(roc_auc)

0.8392213114754098


In [65]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')
print(scores)
print(scores.mean())

[0.83922131 0.80344968 0.7978429  0.80622951 0.77672751]
0.8046941805236123


я так и не понял как для ласт таски сохранить файл ((


In [66]:
preds_proba = model.predict_proba(test)[:, 1]
pd.Series(preds_proba).to_csv('result.txt', index=False, header=False)